In [2]:
import pandas as pd
from IPython.display import HTML
def printmd(string):
    from IPython.display import Markdown, display
    display(Markdown(string))

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### postgresql + python ###
import psycopg2 as pg2

class psql_py:
    def __init__(self):
        self.conn = self.connect_db()
        self.cur = self.conn.cursor()
        self.cur.execute("set search_path = mimiciii;") 
        
    def connect_db(self):
        try:
            conn = pg2.connect("host = localhost dbname=mimic user=postgres port=5432")
            #print('database connected')
            return conn
        except:
            print("I am unable to connect to the database")          
            
    def exe_sql (self, exe_sql_str):
        return pd.read_sql(exe_sql_str,self.conn)
    
    def db_query(self, exe_sql_str, mode='sql'):
        if mode == 'pd':
            return_pd = self.exe_sql (exe_sql_str)
            self.close_db()
            return return_pd
        else: 
            self.cur.execute(exe_sql_str)
            self.close_db()
            
    def close_db (self):
        self.cur.close()
        # commit the changes
        self.conn.commit()
        #print('database disconnected')


In [20]:

printmd('## Population Based on [mimic-code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation-durations.sql) \n' +
            '- Patients using a ventilation extracted by [code](https://github.com/yyjo0430/mimiciii_extubation/tree/master/sql)'
       )

db = psql_py() 
str_sql = 'select * from ventdurations'
a = db.db_query(str_sql,'pd')
total = len(a)
printmd('#### Population: '+ str(total))

## Population Based on [mimic-code](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation-durations.sql) 
- Patients using a ventilation extracted by [code](https://github.com/yyjo0430/mimiciii_extubation/tree/master/sql)

#### Population: 38465

   - The ventilation number (*ventnum*) means the sequence number of use of the single mechanical ventilation 
    
        - The use of the single mechanical ventilation is defined as a set of records where the interval time between adjacent two records is not over 8 hours


In [12]:
printmd('### Exclusion 1\n' +
'- 신호 이상치 제거 \n\
    - extubation, self-extubation, oxygentherapy가 명시적으로 적혀있지 않는 경우 \n\
    - vent 신호만 있는 경우'
)

db = psql_py() 
str_sql = 'select * from ptsinfo_vent'
case1 = db.db_query(str_sql,'pd')
case1 = case1.sort_values(['icustay_id','ventnum','vent_on_time','vent_off_time']).reset_index(drop=True)
case1 = case1.dropna(subset=['icustay_id'])
case1.icustay_id = case1.icustay_id.astype(int)
num_case1 = len(case1)
print('Population: ',num_case1, ', excluded [',total-num_case1,']')
display(case1.sample(5))

### Exclusion 1
- 신호 이상치 제거 
    - extubation, self-extubation, oxygentherapy가 명시적으로 적혀있지 않는 경우 
    - vent 신호만 있는 경우

Population:  33040 , excluded [ 5425 ]


,icustay_id,hadm_id,subject_id,gender,birth_time,extu_age,admittime,ventnum,vent_on_time,vent_off_time,duration_hours,unplanned,tra_used,trache_time,dischtime,death_time,death
8520,225617,184942,16106,F,2039-10-01,85.34,2125-01-31 05:44:00,1,2125-02-01 22:19:00,2125-02-06 11:24:00,109.083333,0,0,NaT,2125-02-09 13:41:00,2129-09-21,0
13519,240823,176342,53435,M,2116-12-10,83.54,2200-06-24 14:35:00,1,2200-06-24 17:00:00,2200-06-29 09:00:00,112.000000,0,0,NaT,2200-07-22 22:05:00,2200-07-22,0
13391,240393,199224,64350,M,2119-09-22,51.81,2171-07-16 21:22:00,1,2171-07-16 21:00:00,2171-07-19 14:44:00,65.733333,0,0,NaT,2171-07-27 14:00:00,NaT,0
18152,255074,166280,59507,F,2049-01-31,77.53,2126-08-13 04:09:00,1,2126-08-14 13:00:00,2126-08-15 03:00:00,14.000000,0,0,NaT,2126-09-07 13:55:00,2126-09-12,0
22701,268325,165634,23110,F,2117-08-29,78.48,2196-02-17 15:15:00,1,2196-02-19 15:00:00,2196-02-20 09:00:00,18.000000,0,0,NaT,2196-02-25 12:30:00,NaT,0


In [ ]:
printmd('### Exclusion 2\n' +
'- 나이: 미성년 및 89세 이상 제거 \n'
)
str_sql = 'select * from icustay_info_v3'
case2 = db.db_query(str_sql,'pd')
case2= case2.sort_values(['icustay_id','ventnum','vent_on_time','vent_off_time']).reset_index(drop=True)
case2 = case2.dropna(subset=['icustay_id'])
case2.icustay_id = case2.icustay_id.astype(int)
case2 = case2[(case2.extu_age>15 )& (case2.extu_age<90)].sort_values('icustay_id')
num_case2 = len(case2)
printmd('#### Population: '+ str(num_case2) + ', excluded ['+str(num_case1-num_case2)+']')


In [ ]:
printmd('### Exclusion 3\n' +
'- Tracheostomy: tracheostymy 사용 환자 제거 \n\
    - 이형숙 선생님이 보내주신 것을 기반 \n\
    - itemid: 940,970, 977, 1022, 1120,4603,4441,687, 689, 690,691,692,1832,1855,1948, \
        5646,2877,2993,6151,8287,6375, 7118,7286,7474,7528,8479,6898 \n\
    - 특정 변수 명가진 itemid 제거 \n\
    (itemid = 687 and (value = refused or value = Not Done)) or \
    (itemid = 940 and (value = no or value =  notdone)) or \
    (itemid = 970 and (value = NOTDONE or value = notdone)) or \
    (itemid = 8479 and (value = No or value = Not Applicable))'        
)
case3 = case2.copy()
case3 = case3[case3.trache==0]
num_case3 = len(case3)
printmd('#### Population: '+ str(num_case3) + ', excluded ['+str(num_case2-num_case3)+']')


In [ ]:
printmd('### Exclusion 4\n' +
'- DEATH: vent 사용중 죽은 환자 제거'    
)
case4 = case3.copy()
case4 = case4[case4.death==0]
num_case4 = len(case4)
printmd('#### Population: '+ str(num_case4) + ', excluded ['+str(num_case3-num_case4)+']')


In [ ]:
case4.head()

In [ ]:
printmd('### Exclusion 5\n' +
'- Period: 1시간 이내 사용자 제거 (변경될 수 있음)'    
)
case5 = case4.copy()
case5 = case5[(case5.duration_hours>1)]
num_case5 = len(case5)
printmd('#### Population: '+ str(num_case5) + ', excluded ['+str(num_case4-num_case5)+']')


In [ ]:
printmd(
        '### Exclusion 6\n' +
        '- ventnum starting number ==1'    
)
case6 = case5.groupby('icustay_id').filter(lambda x:x['ventnum'].min()==1)


num_case6 = len(case6)
printmd('#### Population: '+ str(num_case6) + ', excluded ['+str(num_case5-num_case6)+']')

In [ ]:
len(case6.icustay_id.unique())

In [ ]:
case6.reset_index(drop=True).to_csv('./190323_finalized_population_real.csv')